In [9]:
import numpy as np
import os
import mrcfile
from numba import jit

In [10]:
%%time

@jit(nopython=True)
def interpolate(data):
    found = False
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            for k in range(data.shape[2]):
                if (data[i, j, k, 0] > 0):
                    found = True
                    if i % 2 == 0:
                        ref_x = 0
                    else:
                        ref_x = 1

                    if j % 2 == 0:
                        ref_y = 0
                    else:
                        ref_y = 1

                    if k % 2 == 0:
                        ref_z = 0
                    else:
                        ref_z = 1

                    #print(i, j, k)

                if found:
                    break
            if found:
                break
        if found: 
            break              

    #print(ref_x, ref_y, ref_z)
    #print((ref_x + 1) % 2, (ref_y + 1) % 2, (ref_z + 1) % 2)

    #Change a single into the opposite of the reference at a time
    for i in range((ref_x + 1) % 2, data.shape[0]-1, 2):
        for j in range(ref_y, data.shape[1]-1, 2):
            for k in range(ref_z, data.shape[2]-1, 2):
                for l in range(data.shape[3]):
                    if (data[i, j, k, l] == 0):
                        avg = 0
                        avg += data[i+1, j, k, l]
                        avg += data[i-1, j, k, l]
                        avg = avg / float(2)
                        data[i, j, k, l] = avg


    for j in range((ref_y + 1) % 2, data.shape[1]-1, 2):
        for i in range(ref_x, data.shape[0]-1, 2):
            for k in range(ref_z, data.shape[2]-1, 2):
                for l in range(data.shape[3]):
                    if (data[i, j, k, l] == 0):
                        avg = 0
                        avg += data[i, j+1, k, l]
                        avg += data[i, j-1, k, l]
                        avg = avg / float(2)
                        data[i, j, k, l] = avg


    for k in range((ref_z + 1) % 2, data.shape[2]-1, 2):
        for i in range(ref_x, data.shape[0]-1, 2):
            for j in range(ref_y, data.shape[1]-1, 2):
                for l in range(data.shape[3]):
                    if (data[i, j, k, l] == 0):
                        avg = 0
                        avg += data[i, j, k+1, l]
                        avg += data[i, j, k-1, l]
                        avg = avg / float(2)
                        data[i, j, k, l] = avg


    #Change two indices into the opposite of the reference at a time
    for i in range((ref_x + 1) % 2, data.shape[0]-1, 2):
        for j in range((ref_y + 1) % 2, data.shape[1]-1, 2):
            for k in range(ref_z, data.shape[2]-1, 2):
                for l in range(data.shape[3]):
                    if (data[i, j, k, l] == 0):
                        avg = 0
                        avg += data[i+1, j, k, l]
                        avg += data[i-1, j, k, l]
                        avg += data[i, j+1, k, l]
                        avg += data[i, j-1, k, l]
                        avg = avg / float(4)
                        data[i, j, k, l] = avg


    for i in range((ref_x + 1) % 2, data.shape[0]-1, 2):
        for k in range((ref_z + 1) % 2, data.shape[2]-1, 2):
            for j in range(ref_y, data.shape[1]-1, 2):
                for l in range(data.shape[3]):
                    if (data[i, j, k, l] == 0):
                        avg = 0
                        avg += data[i+1, j, k, l]
                        avg += data[i-1, j, k, l]
                        avg += data[i, j, k+1, l]
                        avg += data[i, j, k-1, l]
                        avg = avg / float(4)
                        data[i, j, k, l] = avg


    for j in range((ref_y + 1) % 2, data.shape[1]-1, 2):
        for k in range((ref_z + 1) % 2, data.shape[2]-1, 2):
            for i in range(ref_x, data.shape[0]-1, 2):
                for l in range(data.shape[3]):
                    if (data[i, j, k, l] == 0):
                        avg = 0
                        avg += data[i, j+1, k, l]
                        avg += data[i, j-1, k, l]
                        avg += data[i, j, k+1, l]
                        avg += data[i, j, k-1, l]
                        avg = avg / float(4)
                        data[i, j, k, l] = avg


    #Change all indices into the opposite of the reference
    for i in range((ref_x + 1) % 2, data.shape[0]-1, 2):
        for j in range((ref_y + 1) % 2, data.shape[1]-1, 2):
            for k in range((ref_z + 1) % 2, data.shape[2]-1, 2):
                for l in range(data.shape[3]):
                    if (data[i, j, k, l] == 0):
                        avg = 0
                        avg += data[i+1, j, k, l]
                        avg += data[i-1, j, k, l]
                        avg += data[i, j+1, k, l]
                        avg += data[i, j-1, k, l]
                        avg += data[i, j, k+1, l]
                        avg += data[i, j, k-1, l]
                        avg = avg / float(6)
                        data[i, j, k, l] = avg

CPU times: user 1.61 ms, sys: 4.66 ms, total: 6.27 ms
Wall time: 6.91 ms


Specify the name of the source map and the name to save the interpolated map:

In [11]:
%%time

source = 'emd_9515_prediction.npy'
to_save = 'interpolated_prediction_9515'

orig_data = np.load(source)
np.set_printoptions(precision=3)

print(orig_data.shape[0])
print(orig_data.shape[1])
print(orig_data.shape[2])
print(orig_data.shape[3])
print(np.count_nonzero(orig_data))

interpolate(orig_data)

np.save(to_save, orig_data)

337
337
337
4
2675096
CPU times: user 4.62 s, sys: 3.95 s, total: 8.57 s
Wall time: 9.76 s


Optionally, you can graph each probability map constructed using the following function:

In [12]:
def conv_prediction_to_mrc(arr,reffile,keyidx,outmapfile):      
    with mrcfile.open(reffile) as mrc:
        nx,ny,nz = arr.shape[0], arr.shape[1], arr.shape[2]
        mx,my,mz,cella = mrc.header.mx,mrc.header.my,mrc.header.mz, mrc.header.cella
        
        
        mrc_new = mrcfile.new(outmapfile,overwrite=True)
        
        mrc_new.set_data(np.zeros((nz, ny, nx), dtype=np.float32))
        mrc_new.header.nxstart=arr.shape[0]
        mrc_new.header.nystart=arr.shape[1]
        mrc_new.header.nzstart=arr.shape[2]
        mrc_new.header.origin.x = mrc.header.origin.x
        mrc_new.header.origin.y = mrc.header.origin.y
        mrc_new.header.origin.z = mrc.header.origin.z
        mrc_new.header.cella['x'] = cella['x']
        mrc_new.header.cella['y'] = cella['y']
        mrc_new.header.cella['z'] = cella['z']
        
        print(nx, ny, nz)
        for i in range(nx - 2):
            for j in range(ny - 2):
                for k in range(nz - 2):
                    mrc_new.data[k, j, i] = arr[i, j, k, keyidx]
        
        vsize=mrc_new.voxel_size
        vsize.flags.writeable = True
        mrc_new.voxel_size=vsize         
        
        mrc_new.update_header_stats()
        
        print("original", mrc.voxel_size)
        mrc.print_header()

        print()
        
        print("new", mrc_new.voxel_size)
        mrc_new.print_header()

        print()
        
        mrc_new.close()

Specify the name of the original .mrc file (with 1 voxel spacing) used to produce probability maps with Emap2sec+ and the name to save the maps. In the naming scheme used:
0 -> Coil
1 -> Beta strand
2 -> Alpha helix
3 -> DNA/RNA

In [13]:
%%time

interpolated = np.load(to_save + '.npy')
mrc = 'emd_9515.mrc'
i_mrc = 'interpolated_prediction_9515'

ids = [0, 1, 2, 3]
for i in ids:
    if i == 0:
        identifier = "_0.mrc" 
    elif i == 1:
        identifier = "_1.mrc"
    elif i == 2:
        identifier = "_2.mrc"
    else:
        identifier = "_3.mrc"
      
    conv_prediction_to_mrc(interpolated, mrc, i, i_mrc + identifier)

337 337 337
original (1., 1., 1.)
nx              : 337
ny              : 337
nz              : 337
mode            : 2
nxstart         : 0
nystart         : 0
nzstart         : 0
mx              : 337
my              : 337
mz              : 337
cella           : (337., 337., 337.)
cellb           : (90., 90., 90.)
mapc            : 1
mapr            : 2
maps            : 3
dmin            : -0.09175386279821396
dmax            : 0.18545447289943695
dmean           : -5.11524012836162e-05
ispg            : 1
nsymbt          : 0
extra1          : b'\x00\x00\x00\x00\x00\x00\x00\x00'
exttyp          : b''
nversion        : 20140
extra2          : b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
origin  